In [1]:
import numpy as np

K = []
Sgm = []
F, F_dfa = [],[] #其实一个的np，一个是list。。前期没设计好
Z, Z_dfa = [],[]
K_num, F_dfa_num = {},{}

def nfa_input():
    global K,Sgm,F,S,Z
    with open('nfa.txt', 'r') as f:
        lines = f.readlines()
    Sgm = lines[0].split()
    
#     F = [[] for i in range(len(lines)-1)]
    F = np.zeros((len(lines)-1)*len(Sgm), dtype=np.int32).reshape((len(lines)-1),len(Sgm))
    K_num['#'] = -1
    for i in range(1,len(lines)):
        K.append(lines[i][0])
        K_num[K[i-1]] = i-1
        
    for i in range(1,len(lines)):
        for j in range(len(lines[i][1:].split())):
            if len(lines[i][1:].split()[j]) > 1: # 能到多处
                mul = int(''.join([str(K_num[n]) for n in lines[i][1:].split()[j].split('_')]))
                F[i-1,j] = -mul #用于区分
            else:
                F[i-1,j] = K_num[lines[i][1:].split()[j]]
#     z = input("请输入终态：（按逗空格分隔）")
#     Z = [K_num[zn] for zn in z.split()]
    Z = [10]
nfa_input()
K, K_num, Sgm, F, Z

(['X', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'Y'],
 {'#': -1,
  'A': 1,
  'B': 2,
  'C': 3,
  'D': 4,
  'E': 5,
  'F': 6,
  'G': 7,
  'H': 8,
  'I': 9,
  'X': 0,
  'Y': 10},
 ['eps', 'a', 'b'],
 array([[  -1,   -1,    1],
        [-265,   -1,   -1],
        [   4,    3,   -1],
        [  -1,   -1,    4],
        [ -25,   -1,   -1],
        [   1,    9,   -1],
        [  -1,   -1,    7],
        [  -1,   -1,    8],
        [   5,   -1,   -1],
        [  -1,   -1,   10],
        [  -1,   -1,   -1]]),
 [10])

In [2]:
import itertools

F_dfa = []
Z_dfa = []
F_dfa_num = {}
def ns2n(n):
    if n < -1:
        n = -n
        m = []
        while n > 0:
            m.append(str(n%10))
            n //= 10 
        return [int(mm) for mm in m]#np.array(m, dtype=np.int32)
    elif n == -1: #因为闭包函数的m接收list
        return []
    else:
        return [n]
    

def e_closure(K): # 传入状态集，不申明global可以也用K表示
    global II #不写会不改变II，递归无法进行
    for k in K:
        n = F[k][0]
        m = ns2n(n) 
        II.add(k)
        if len(m) > 0:
            for n in m:            
                if n not in II:
                    II.add(n)        
                    II = II.union(e_closure([n]))           
    return II

def move(k, sgm): #传入状态和符号
    n = F[k, sgm] 
    return ns2n(n) #状态集
    

pre_I_queue = [] #新的 DFA集合，即将要访问的集合
I_queue = [] # DFA 集合队列

II = set() #求闭包前都要初始化
I = e_closure({0}) #一个set
pre_I_queue.append(I)
I_queue.append(I)


def nfa_to_dfa():
    global I,II,K_dfa,F_dfa,Z_dfa,F_dfa_num
    I_dfa_num = 0
    F_dfa_num['set()'] = -1
    
    while 1:       
        if len(pre_I_queue) == 0:
            break
        else:
            I = pre_I_queue[0]
            F_dfa_num[str(I)] = I_dfa_num #建立新集合的字典
            I_dfa_num += 1
            for i,z in itertools.product(I,Z):#添加终态集
                if i == z and I not in Z_dfa:
                    Z_dfa.append(str(I))
                    break
            print('%-25s'%(str(I)), end='')
            
            for sgm in range(1,len(Sgm)):
                I_next = set()
#                 print(" SGM:", sgm, end=',')
                for i in I:
                    II = set()
                    e_move = e_closure(move(i,sgm))
                    I_next = I_next.union(e_move)
                F_dfa.append(str(I_next))
                print('%-10s'%(str(I_next)), end='')
                if I_next not in I_queue and len(I_next)!=0:
                    I_queue.append(I_next)
                    pre_I_queue.append(I_next)
                    
                    
            pre_I_queue.pop(0) 
            print()
    K_dfa = {i for i in range(len(I_queue))}
    F_dfa = [F_dfa_num[i] for i in F_dfa]
    F_dfa = np.array(F_dfa).reshape(len(I_queue),-1)
    Z_dfa = {F_dfa_num[i] for i in Z_dfa}
nfa_to_dfa()
print('\nK_dfa：\n',K_dfa,'\n\nF_dfa：\n',F_dfa,'\n\n终态：\n',Z_dfa)

{0}                      set()     {1, 2, 4, 5, 6}
{1, 2, 4, 5, 6}          {9, 3}    {7}       
{9, 3}                   set()     {1, 2, 4, 5, 6, 10}
{7}                      set()     {1, 2, 4, 5, 6, 8}
{1, 2, 4, 5, 6, 10}      {9, 3}    {7}       
{1, 2, 4, 5, 6, 8}       {9, 3}    {7}       

K_dfa：
 {0, 1, 2, 3, 4, 5} 

F_dfa：
 [[-1  1]
 [ 2  3]
 [-1  4]
 [-1  5]
 [ 2  3]
 [ 2  3]] 

终态：
 {4}


In [3]:
S = set()
P = [K_dfa-Z_dfa, Z_dfa] #第一次划分
P_next = [K_dfa-Z_dfa, Z_dfa] #！！！不能写P_next = P，不然改变P_next时，P也会一起改变
flag_P = 1
n=1
def min_dfa():
    global P,flag_P,S,n
    while 1:
        if flag_P == 0:
            break
        flag_P = 0 #判断所有集合是否可以分割
        P.clear() #不能写 P = P_next！也会死循环！！
        P.extend([p for p in P_next if len(p)!=0])     
        P_next.clear()
        P_next.extend([p for p in P if len(p)!=0])     
        
        for p in P: #每个划分   
            flag_p = 0 #判断该集合是否可以分割
            for sgm in range(len(Sgm)-1):#因为最初的还有eps，要减去    
                S = set()
                for k in p: #一个划分中的每个元素
                    if F_dfa[k,sgm] not in p and F_dfa[k,sgm] != -1:
                        """
                        是不在一个同一个集合里！两个for
                        """
                        S.add(k)
                
                if len(S) != 0: # 该p可分
                    print('aaa',S)
                    flag_P = 1
                    
                    if len(p-S) != 0:
                        flag_p = 1
                        P_next.extend([S,p-S])
                    print(P_next,sgm,p-S)
            if flag_p == 1 :
                P_next.remove(p)
    
min_dfa()
print(P_next)

aaa {2}
[{0, 1, 2, 3, 5}, {4}, {2}, {0, 1, 3, 5}] 1 {0, 1, 3, 5}
aaa {4}
[{4}, {2}, {0, 1, 3, 5}] 0 set()
aaa {4}
[{4}, {2}, {0, 1, 3, 5}] 1 set()
aaa {4}
[{4}, {2}, {0, 1, 3, 5}] 0 set()
aaa {4}
[{4}, {2}, {0, 1, 3, 5}] 1 set()
aaa {2}
[{4}, {2}, {0, 1, 3, 5}] 1 set()
aaa {1, 5}
[{4}, {2}, {0, 1, 3, 5}, {1, 5}, {0, 3}] 0 {0, 3}
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1

aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1

aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1

[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5},

[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3

...

[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5},

aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1

[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3

[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {0, 3}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {4}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {2}
[{4}, {2}, {1, 5}, {0, 3}] 1 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0, 3}] 0 set()
aaa {1, 5}
[{4}, {2}, {1, 5}, {0

KeyboardInterrupt: 

In [51]:
a = [1,2]
a.append("1,2")
a[-1]= a[-1].split(',')

a = [1,2]
Z = [2,3]
z = []
z = [str(a) for i in a for j in Z if i==j]
a = '{1,2}'
set(a)

{',', '1', '2', '{', '}'}

# GUI

In [ ]:
global stack
 
class State:
    #counter = 0
    def __init__(self,c,out = None,out1 = None):
        self.c = c
        self.out = out
        self.out1 = out1
        self.lastlist = 0
        #State.counter += 1  
        
class Fragment:
    def __init__(self,begin,end = None):
        self.begin = begin
        self.end = end
 
_BEGIN = '^'  
_NULL = ' '  
def make_begin():  
    return Fragment(State(_BEGIN))
def make_null():
    return Fragment(State(_NULL))
 
_END = '$'
def make_end():
    return Fragment(State(_END)) 
 
def make_atom(char):  
    a = State(char)
    print ("make_atom",a.__class__)
    return Fragment(a,a)  
 
def make_seq(frag1, frag2):  
    if frag2 is None: return frag1  
    else:  
        #print frag1.end.__class__
        frag1.end.out = frag2.begin  
        #frag1.end = frag2.begin  
        return Fragment(frag1.begin, frag2.end)
 
_Split = '`'   
_MERGE = '!'     
def SplitState(state1,state2):
    return State(_Split,out = state1,out1 = state2)
def MergeState(state1,state2):
    a = State(_MERGE)
    print (state1.__class__)
    state1.out = a
    state2.out = a
    return a
 
def make_alt(frag1,frag2):  
    split = SplitState(frag1.begin, frag2.begin)  
    merge = MergeState(frag1.end,frag2.end)  
    return Fragment(split, merge)   
 
    
def build(string):
    global stack,k
    alpha = "abcdefghijklmnopqrstuvwxyz"
    c = string 
    if c == '$':
        e = make_end()
        k = k - 1
        stack[k] = make_seq(stack[k],e)
        print (k)
    elif c in alpha :
        stack[k] = make_atom(c)
        k = k + 1
    elif c == '|':
        k = k - 1
        e1 = stack[k]
        k = k - 1
        e2 = stack[k]
        stack[k] = make_alt(e1,e2)
        k = k + 1
    elif c == '.':
        k = k - 1
        e1 = stack[k]
        k = k - 1 
        e2 = stack[k]
        e1 = make_seq(e2,e1)
        stack[k] = e1
        k = k + 1
    elif c == '?':
        k = k - 1
        e1 = stack[k]
        split_state = State(_Split,out = e1.begin)
        merge_state = State(_MERGE)
        split_state.out1 = merge_state
        e1.end.out = merge_state
        stack[k] = Fragment(split_state,merge_state)
        k = k + 1
    elif c == '*':
        k = k - 1
        e1 = stack[k]
        split_state = State(_Split,out = e1.begin)
        merge_state = State(_MERGE)
        split_state.out1 = merge_state
        e1.end.out = split_state
        stack[k] = Fragment(split_state,merge_state)
        k = k + 1
        
        
def post2nfa(string):
    global stack,k
    stack = []
    k = 0
    for l in range(1000):
        stack.append(make_null())
        
    for i in string:
        build(i)
    
def match(string):
    global stack
    nfa = stack[0]
    curr_set = { nfa.begin }
    match_pos = []
    i = 0
    while i < len(string):
        match_set = set()
        while curr_set:
            s = curr_set.pop()
            if s.c == _BEGIN or s.c == _MERGE:
                curr_set.add(s.out)
                #print "match ",str(s.c),"out:",str(s.out.c)
            elif s.c == _Split:
                curr_set.add(s.out)
                curr_set.add(s.out1)
                #print "add",s.out1.c,s.out.c
            elif s.c == string[i]:
                match_set.add(s.out)
                print ("match char",str(s.c),"next",str(s.out.c))
                if s.out.c == _END:
                    match_pos.append(i)
                    print ("matched",i)
            #print "curr_set:",
            #for s in curr_set:
            #    print s.c,
            else:
                #print "pass "
                continue
            
        if match_set:
            curr_set = { x for x in match_set}
            print ("match set ",)
            for z in match_set:
                print (z.c,)
            i = i + 1
        else:
            return match_pos
    return match_pos
post2nfa("a*")
                
    
